In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import scipy.linalg as la
from scipy.stats import linregress, spearmanr, kendalltau, pearsonr
import sys
sys.path.append('..')
from draw_network import draw_network
import re

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import matplotlib
matplotlib.rc('xtick',labelsize=22)
matplotlib.rc('ytick',labelsize=16)
matplotlib.rc('font',size = 24)
matplotlib.rc('legend',fontsize = 17)
matplotlib.rc('figure',titlesize = 25)

In [2]:
def initialize_df():
    
    # import and clean the dataframe with alliances and conflicts
    
    df = pd.read_csv('../CoW_data/allies_and_enemies_1816_2014_iso.csv')

    df = df[(df['alliance'] != 0) | (df['conflict']!=0)]   #filter entries with no link
    
    df['weight'] = df['alliance']+df['conflict']+df['alliance']*df['conflict']   # merge alliances and conflicts in a single column
    df = df.drop(columns = ['alliance','conflict'])
    return df



def create_network(df, year, giant_component = True):
    
    # returns a Graph object corresponding to the international relations network of the given year
    # edges have a weighted attribute which can be 1 or -1
    
    df2 = df[df['year']==year]   # select one particular year
    G = nx.from_pandas_edgelist(df2, source ='statea', target = 'stateb', edge_attr='weight')
    
    # take only the giant component
    if giant_component:
        GC = max(nx.connected_components(G), key=len)
        G = G.subgraph(GC)
    return G


def local_balance(G, country):   
    
    ### find node balance of a given country for one specific graph (i.e. one year)   ###
    # find the balance index of the specific country
    
    idx = np.where(np.array(G.nodes()) == country)[0][0]    
    
    # calculate signed communicability
    A = nx.adjacency_matrix(G).todense()
    A0 = np.abs(A)
    
    Comm = la.expm(A)
    Comm0 = la.expm(A0)
    
    # calculate balance
    Ki = np.diag(Comm)/np.diag(Comm0)       # node balance
    
    return Ki[idx]




def local_balance_timeseries(country, year_range = [1816,2014]):
    
    # finds the local balance for several years and creates a balance time series for a specific country
    # Input:
    # country: string with the iso-3 code of the country for which to compute the local balance time series
    # year_range (optional): list with the first and last year of the time series
    # Output:
    # years_v: 1D array with the years of the time series
    # Kloc: 1D array with the local balance values for each year
    
    df = initialize_df()
    years_v = np.arange(year_range[0],year_range[1]+1)
    Kloc = np.zeros(len(years_v))
    
    for i, year in enumerate(years_v):

        G = create_network(df, year)
        if country not in G.nodes():
            Kloc[i] = None
        else:
            Kloc[i] = local_balance(G, country)

    return years_v, Kloc


def randomize_timeseries(time_series):

    # Randomize a time series by shuffling its values
    # Input:
    # time_series: 1D array of values
    # Output:
    # time_series_copy: 1D array of values, shuffled

    # Shuffle only the non-NaN values
    time_series_copy = time_series.copy()   # note for myself: I need to make a copy because otherwise the original time series gets randomized outside this function too
    mask_non_nan = ~np.isnan(time_series_copy)
    shuffled_non_nan = time_series_copy[mask_non_nan]
    np.random.shuffle(shuffled_non_nan)

    # Place the shuffled non-NaN values back into the original time series using the mask
    time_series_copy[mask_non_nan] = shuffled_non_nan

    return time_series_copy


# Comparison with GPR index

In [3]:

def compute_correlation(timeseries_1, timeseries_2, type = 'spearman'):
    
    # computes the correlation between two time series
    # Input:
    # timeseries_1: 1D array of values
    # timeseries_2: 1D array of values
    # type: string, type of correlation to compute. Can be 'spearman', 'kendall' or 'pearson'
    # Output:
    # corr: correlation coefficient value
    # p_value: p-value of the correlation

    # filter nan values
    timeseries_1 = timeseries_1.dropna()
    timeseries_2 = timeseries_2.dropna()

    # keep only the years common to both time series
    overlap_index = timeseries_1.index.intersection(timeseries_2.index)
    timeseries_1_ovlap = timeseries_1.loc[overlap_index]
    timeseries_2_ovlap = timeseries_2.loc[overlap_index]

    # compute correlations
    if type == 'spearman':
        corr, p_value = spearmanr(timeseries_1_ovlap.values, timeseries_2_ovlap.values)

    elif type == 'kendall':
        corr, p_value = kendalltau(timeseries_1_ovlap.values, timeseries_2_ovlap.values)

    elif type == 'pearson':
        corr, p_value = pearsonr(timeseries_1_ovlap.values, timeseries_2_ovlap.values)
        
    else:
        raise ValueError('Only spearman, kendall or pearson correlation allowed')

    return corr, p_value

In [4]:
### Cleaning the GPR dataset ###

# import GPR df
GPR_df = pd.read_excel('GPR_data/GPR_individual_countries.xls')

# select only the country GPR columns
columns_to_use = list(filter(lambda col: col.startswith('GPRHC_'), GPR_df.columns))
columns_to_use.insert(0, 'month')
GPR_df = GPR_df[columns_to_use]

# average the GPR every year
GPR_df['Year'] = GPR_df['month'].dt.year 
GPR_year = GPR_df.groupby(by = 'Year').mean()  



### filter countries for which there is sufficient data  ####

countries = [item.replace('GPRHC_', '') for item in GPR_year.columns]

filtered_countries = []
for i, country in enumerate(countries):

    # get balance index time series
    years_v, Kloc = local_balance_timeseries(country)

    # filter countries for which there is sufficient data
    years_filtered = years_v[~np.isnan(Kloc)]
    overlapping_years = set(years_filtered).intersection(set(np.arange(1900,2014)))
    if len(overlapping_years) < 100:  # at least 100 data points per country
        continue 
    else:
        filtered_countries.append(country)




In [7]:
M = 500   # number of randomizations
corr_type = 'spearman'
prefix = 'GPRHC_'

# initialize stuff
corr_true = np.zeros(len(filtered_countries));     # correlation for the true time series
p_values_true = np.zeros(len(filtered_countries));  # p-value for the true time series

corr_rand_avg = np.zeros(len(filtered_countries))      # avg correlation for randomized time series
p_values_rand_avg = np.zeros(len(filtered_countries))   # avg p-value for randomized time series

for i, country in enumerate(filtered_countries):

    GPR_ts = GPR_year[prefix+country]      # get GPR index time series

    # get balance index time series and compute correlations
    years_v, Kloc = local_balance_timeseries(country)
    Kloc_ts = pd.Series(Kloc, index = years_v)
    corr_true[i], p_values_true[i] = compute_correlation(GPR_ts, Kloc_ts, type = corr_type)

    # correlations for the randomized time series
    for _ in range(M):
        # randomize time series
        Kloc_aux = np.copy(Kloc)
        Kloc_rand = randomize_timeseries(Kloc_aux)

        # compute correlations
        Kloc_ts_rand = pd.Series(Kloc_rand, index = years_v)
        corr_rand, p_value_rand = compute_correlation(GPR_ts, Kloc_ts_rand, type = corr_type)
        corr_rand_avg[i] += corr_rand
        p_values_rand_avg[i] += p_value_rand

    corr_rand_avg /= M
    p_values_rand_avg /= M

    print(country,':')
    print('Correlation true: ', corr_true[i])
    print('P-value true: ', p_values_true[i])
    print('Correlation random: ', corr_rand_avg[i])
    print('P-value random: ', p_values_rand_avg[i])
    print('---')


CHN :
Correlation true:  -0.16120886924958572
P-value true:  0.08522115149453217
Correlation random:  0.006982590529118299
P-value random:  0.5054482518790696
---
FRA :
Correlation true:  -0.4138069297908103
P-value true:  4.730014550349174e-06
Correlation random:  0.004048813866121486
P-value random:  0.4990116521381114
---
GBR :
Correlation true:  -0.18540203582419318
P-value true:  0.04728602936260216
Correlation random:  -0.0022038994461457307
P-value random:  0.516602438925751
---
ITA :
Correlation true:  -0.23730766195849443
P-value true:  0.0106607036992213
Correlation random:  0.00400034173259127
P-value random:  0.4982353398872249
---
JPN :
Correlation true:  -0.10930404967102217
P-value true:  0.2578846123064046
Correlation random:  0.009357446261629475
P-value random:  0.5043816831648504
---
PRT :
Correlation true:  -0.3668902390909809
P-value true:  5.5056821147762176e-05
Correlation random:  0.008116640962975423
P-value random:  0.4836953155326624
---
RUS :
Correlation tru